In [3]:
import skimage.io
import dask.array as da
import dask
from pathlib import Path
import napari
from natsort import natsorted
import numpy as np
from napari_flim_phasor_calculator._reader import get_current_tz


Define read 2d and 3d functions

In [4]:

folder_path = r"C:\Users\mazo260d\Desktop\Conni_BiA_PoL\stack_as_tif"
folder_path = Path(folder_path)

def read_tif_data_2D(path):
    from skimage.io import imread
    image = imread(path)
    return image

def read_tif_data_3D(file_paths, image_slice_max_shape):
    file_paths = natsorted(file_paths)
    z_list = []
    for file_path in file_paths:
        if file_path.suffix == '.tif':
            current_t, current_z = get_current_tz(file_path)
            data = read_tif_data_2D(file_path)
            if current_z is not None:
                image = np.zeros(image_slice_max_shape, dtype=data.dtype)
                image[:data.shape[0], :data.shape[1], :data.shape[2], :data.shape[3]] = data
                z_list.append(image)
    image_3D = np.stack(z_list)
    # move channel and microtime to the beginning (putting z behind them): from (z, ch, mt, y, x) to (ch, mt, z, y, x)
    image_3D = np.moveaxis(image_3D, [-4, -3], [0, 1])
    return image_3D

Get max slice shape

In [5]:
# Read all slices to get slice max shape and dtype (not ideal but for now it is OK)
slice_shape_list = []
for file_path in folder_path.iterdir():
    if file_path.suffix == '.tif':
        image_2D = read_tif_data_2D(file_path)
        slice_shape_list.append(image_2D.shape)
slice_max_shape = max(slice_shape_list)
print('last_slice_shape = ', image_2D.shape,'image_dtype = ', image_2D.dtype)
print('max_slice_shape = ', slice_max_shape)

last_slice_shape =  (2, 266, 512, 512) image_dtype =  uint16
max_slice_shape =  (2, 276, 512, 512)


Get max z slices and max t timepoints from filepaths

In [6]:
file_paths = [file_path for file_path in folder_path.iterdir() if file_path.suffix == '.tif']

# Get max z slices by reading all file names
def get_max_zslices(file_paths):
    max_z = max([get_current_tz(file_path) for file_path in file_paths if file_path.suffix == '.tif'])[1]
    if max_z is None:
        return 1
    return max_z
def get_max_time(file_paths):
    max_time = max([get_current_tz(file_path) for file_path in file_paths if file_path.suffix == '.tif'])[0]
    if max_time is None:
        return 1
    return max_time

max_z = get_max_zslices(file_paths)
print('max_z slices = ', max_z)
max_t = get_max_time(file_paths)
print('max_time = ', max_t)

max_z slices =  65
max_time =  1


Make dask array

In [7]:
# Using map_blocks

def read_one_image(block_id, filenames=file_paths, axis=0):
    # a function that reads in one chunk of data
    path = filenames[block_id[axis]]
    image = read_tif_data_2D(path)
    return np.expand_dims(image, axis=axis)

# load the first image (assume rest are same shape/dtype)
sample = read_tif_data_2D(file_paths[0])

stack = da.map_blocks(
    read_one_image,
    dtype=sample.dtype,
    chunks=((1,) * len(file_paths),  *slice_max_shape)
)
stack


dask.array<read_one_image, shape=(65, 2, 276, 512, 512), dtype=uint16, chunksize=(1, 2, 276, 512, 512), chunktype=numpy.ndarray>

In [8]:

stack = stack.rechunk((max_z, *slice_max_shape))
stack

dask.array<rechunk-merge, shape=(65, 2, 276, 512, 512), dtype=uint16, chunksize=(65, 2, 276, 512, 512), chunktype=numpy.ndarray>